In [3]:
import collections
import copy
import cspUtil
import dataUtil
import itertools
import numpy as np
import matplotlib.pyplot as plt
import os

In [4]:
def countSequences(inputList, N, gap=0, domain=None, laplace=1):
    out = collections.defaultdict(float)
    if gap == 0:
        for i in range(len(inputList)-N+1):
            if [inputList[i+k] != None for k in range(N)] == [True for _ in range(N)]:
                seq = tuple([inputList[i+k] for k in range(N)])
                out[seq] += 1
        #print out
    else:
        assert N == 2
        for i in range(len(inputList)-N+1-gap):
            if (inputList[i] != None) and (inputList[i+gap+1] != None):
                seq = tuple([inputList[i], inputList[i+gap+1]])
                out[seq] += 1
                
    # add Laplace smoothing
    if domain is not None:
        for val in domain:
            out[val] += laplace
        

    factor = 1.0 / sum(out.itervalues())
    for k in out:
          out[k] *= 10 * factor

    return out

In [5]:
def printBinaryFactor(binaryFactor):
    for var1, dict1 in binaryFactor.items():
        for var2, dict2 in dict1.items():
            for val1, dict3 in dict2.items():
                for val2, weight in dict3.items():
                    if weight:
                        print var1, val1, var2, val2, weight
                        
def printUnaryFactor(unaryFactor):
    for var1, dict1 in unaryFactor.items():
        if dict1 is None: continue
        for val1, weight in dict1.items():
            if weight:
                print var1, val1, weight

In [6]:
def createRythmCSP(windowedPrimTrack, window, bi=True, tri=False, bi_gap=True):
    cspRythm = cspUtil.CSP()
    (start, end) = window
    # Creating "units" variables
    transitions = [note[1] for note in windowedPrimTrack]
    for unitNum in range(start, end+1):
        unit = windowedPrimTrack[unitNum]
        cspRythm.add_variable(('U', unitNum), [(unit[0],True),(unit[0],False)])
    
    if bi:
        # Create binary factors proportional to frequence of apparition of pairs
        biSequences = countSequences(transitions, 2)
        for varNum in range(start, end):
            cspRythm.add_binary_factor(('U', varNum), ('U', varNum+1), lambda u1, u2 : biSequences[(u1[1], u2[1])])
        ###### Add corner cases
        cspRythm.add_unary_factor(('U', start), lambda u : biSequences[(windowedPrimTrack[start-1][1], u[1])])
        cspRythm.add_unary_factor(('U', end)  , lambda u : biSequences[(u[1], windowedPrimTrack[end+1][1])])
     
    if bi_gap:  
        # Create binary factors with gap
        for gap in range(1,len(window)):
            biSequencesGap = countSequences(transitions, 2, gap)
            for varNum in range(start, end-gap):
                cspRythm.add_binary_factor(('U', varNum), ('U', varNum+1+gap), lambda u1, u2 : biSequencesGap[(u1[1], u2[1])])
            ###### Add corner cases
            for varNum in range(start, start+gap):
                cspRythm.add_unary_factor(('U', varNum), lambda u : biSequencesGap[(windowedPrimTrack[varNum-1-gap][1], u[1])])
            for varNum in range(end-gap, end): 
                cspRythm.add_unary_factor(('U', varNum)  , lambda u : biSequencesGap[(u[1], windowedPrimTrack[varNum+2+gap][1])])

    if tri:
        ### Compute domain for variables which represent two consecutive pitches ###
        domain2 = tuple(itertools.product((True,False), repeat = 2))

        ### Create variables which represent 2 consecutive pitches as ('B', pitchIdx) ###
        for rythmIdx in range(start, end):
            cspRythm.add_variable(('B', rythmIdx), copy.deepcopy(domain2))
                                                                                                                                                                                                     # Creating "binary" variables for 3-sequences handling
        ### Creating binary factors proportional to frequence of apparition of triads ###                                                                                                                               triSequences = countSequences(windowedPrimTrack, 3)
        freq3 = countSequences(transitions, 3)
        for rythmIdx in range(start, end):
            # Add constraint B_i[0] = U_i
            cspRythm.add_binary_factor(('U', rythmIdx), ('B', rythmIdx), lambda u, b: u[1] == b[0])

            if rythmIdx < end-1:
                # Add consitency contraints B_i[0] = B_{i+1}[1] and frequency factor
                cspRythm.add_binary_factor(('B', rythmIdx), ('B', rythmIdx+1), \
                                        lambda b1, b2: (b1[0] == b2[1]) * freq3[(b1[1], b1[0], b2[0])]) 
                
            
        # Add corner cases
        cspRythm.add_unary_factor(('B', start), lambda b: (b[1] == transitions[start-1]) \
                          * freq3[(transitions[start-2], transitions[start-1], b[0])])
        cspRythm.add_unary_factor(('B', end-1), lambda b: freq3[(b[1], b[0], transitions[end+1])] \
                     * freq3[(b[0], transitions[end+1], transitions[end+2])])

    return cspRythm

In [7]:
def createPitchCSP(windowedPitchPrimTrack, weight_uni, weight_bi, weight_tri, window, bi=True, tri=False, bi_gap=False, laplace=0):
    (start, end) = window
    cspPitch = cspUtil.CSP()
    pitchTrack = [note[0] for note in windowedPitchPrimTrack]
    # Creating "units" variables #
    # Domains
    minPitch, maxPitch = pitchTrack[0], pitchTrack[0]
    for pitch in pitchTrack:
        if type(pitch) == int:
            minPitch, maxPitch = min(minPitch, pitch), max(maxPitch, pitch)
    domains = tuple(list(range(minPitch, maxPitch+1)) + ['silence'])
    
    for pitchNum in range(start, end+1):
        cspPitch.add_variable(('U', pitchNum), copy.deepcopy(domains))

     #frequency of single apparition
    freq1 = countSequences(pitchTrack, 1, 0, domains, laplace)
    for pitchIdx in range(start, end):
        cspPitch.add_unary_factor(('U', pitchIdx), lambda p : freq1[(p,)]**weight_uni)
        
    # Create binary factor: must keep the same pitch if previous duration = True
    for varNum in range(start, end): 
        if windowedPitchPrimTrack[varNum][1]:
            cspPitch.add_binary_factor(('U', varNum), ('U', varNum+1), lambda x, y : x == y)
    ###### Add corner cases
    if windowedPitchPrimTrack[start-1][1]:
        cspPitch.add_unary_factor(('U', start), lambda x : x == windowedPitchPrimTrack[start-1][0])
    if windowedPitchPrimTrack[end][1]:
        cspPitch.add_unary_factor(('U', end), lambda x : x == windowedPitchPrimTrack[end+1][0])

    if bi:
        # Create binary factors proportional to frequence of apparition of pairs
        biSequences = countSequences(pitchTrack, 2, 0, tuple(itertools.product(domains, repeat = 2)), laplace)
        for varNum in range(start, end):
            cspPitch.add_binary_factor(('U', varNum), ('U', varNum+1), lambda p1, p2 : biSequences[(p1, p2)]**weight_bi)
        ###### Add corner cases
        cspPitch.add_unary_factor(('U', start), lambda p : biSequences[(pitchTrack[start-1], p)]**weight_bi)
        cspPitch.add_unary_factor(('U', end)  , lambda p : biSequences[(p, pitchTrack[end+1])]**weight_bi)
    
    if bi_gap:
        # Create binary factors with gap
        for gap in range(1,len(window)):
            biSequencesGap = countSequences(pitchTrack, 2, gap)
            for varNum in range(start, end-gap):
                cspPitch.add_binary_factor(('U', varNum), ('U', varNum+1+gap), lambda p1, p2 : biSequencesGap[(p1, p2)]**weight_bi)
            ###### Add corner cases
            for varNum in range(start, start+gap):
                cspPitch.add_unary_factor(('U', varNum), lambda p : biSequencesGap[(pitchTrack[varNum-1-gap], p)]**weight_bi)
            for varNum in range(end-gap, end): 
                cspPitch.add_unary_factor(('U', varNum)  , lambda p : biSequencesGap[(p, pitchTrack[varNum+2+gap])]**weight_bi)

    if tri:
        ### Compute domain for variables which represent two consecutive pitches ###
        domain2 = tuple(itertools.product(domains, repeat = 2))

        ### Create variables which represent 2 consecutive pitches as ('B', pitchIdx) ###
        for pitchIdx in range(start, end):
            cspPitch.add_variable(('B', pitchIdx), copy.deepcopy(domain2))
                                                                                                                                                                                                         # Creating "binary" variables for 3-sequences handling
        ### Creating binary factors proportional to frequence of apparition of triads ###                                                                                                                               triSequences = countSequences(windowedPrimTrack, 3)
        freq3 = countSequences(pitchTrack, 3, 0, tuple(itertools.product(domains, repeat = 3)), laplace)
        for pitchIdx in range(start, end):
            # Add constraint B_i[0] = U_i
            cspPitch.add_binary_factor(('U', pitchIdx), ('B', pitchIdx), lambda u, b: u == b[0])

            if pitchIdx < end-1:
                # Add consitency contraints B_i[0] = B_{i+1}[1] and frequency factor
                cspPitch.add_binary_factor(('B', pitchIdx), ('B', pitchIdx+1), \
                                        lambda b1, b2: (b1[0] == b2[1]) * freq3[(b1[1], b1[0], b2[0])]**weight_tri)
        # Add corner cases
        cspPitch.add_unary_factor(('B', start), lambda b: (b[1] == pitchTrack[start-1]) \
                              * freq3[(pitchTrack[start-2], pitchTrack[start-1], b[0])]**weight_tri)
        cspPitch.add_unary_factor(('B', end-1), lambda b: freq3[(b[1], b[0], pitchTrack[end+1])]**weight_tri \
                         * freq3[(b[0], pitchTrack[end+1], pitchTrack[end+2])]**weight_tri)
    
    #printBinaryFactor(cspPitch.binaryFactors)
        
    return cspPitch

In [8]:
def reconstruction(mypattern,  window, weight_uni, weight_bi, weight_tri, trackNum=0, bi=True, tri=False, rhythmRec=False, bi_gap=False, laplace=0):
    rythmUnit, windowedPrimTrack = mypattern.getCorrupt(window, trackNum)
    windowedPitchPrimTrack = windowedPrimTrack[:]
    if rhythmRec:
        # Rhythm reconstruction
        CSP_rythm = createRythmCSP(windowedPrimTrack, window, bi, tri, bi_gap)
        search_rythm = cspUtil.BacktrackingSearch()
        search_rythm.solve(CSP_rythm, mcv = False, ac3 = False)  
        
        for i, note in enumerate(windowedPitchPrimTrack[:]):
            if note == (None, None):
                windowedPitchPrimTrack[i] = search_rythm.optimalAssignment[('U', i)]
    else:
        # No Rhythm reconstruction (rhythm based on the original score)
        for i, note in enumerate(windowedPitchPrimTrack[:]):
            if note == (None, None):
                windowedPitchPrimTrack[i] = (None, mypattern.primTracks[trackNum][i][1])

    # Pitch reconstruction
    CSP_pitch = createPitchCSP(windowedPitchPrimTrack, weight_uni, weight_bi, weight_tri, window, bi, tri, bi_gap, laplace)
    search_pitch = cspUtil.BacktrackingSearch()
    search_pitch.solve(CSP_pitch, mcv = False, ac3 = False)
    primReconstruction = windowedPitchPrimTrack[:]
    for i, note in enumerate(primReconstruction[:]):
        if note[0] == None:
            if ('U', i) in search_pitch.optimalAssignment.keys():
                primReconstruction[i] = (search_pitch.optimalAssignment[('U', i)], primReconstruction[i][1])
    
    return primReconstruction

In [9]:
def lossFunction(window, primOriginal, primReconstruction):
    (start, end) = window
    accuracy = 0
    for i in range(start, end+1):
        if primOriginal[i][0] == primReconstruction[i][0]:
            accuracy += 1
        else: 
            print 'Position:', i, 'Original pitch:', primOriginal[i][0], 'Reconstructed pitch:', primReconstruction[i][0]
            
        if primOriginal[i][1] == primReconstruction[i][1]:
            accuracy += 1
        else: 
            print 'Position:', i, 'Original rhythm:', primOriginal[i][1], 'Reconstructed rhythm:', primReconstruction[i][1]
            
    loss = 100 * (1 - accuracy/(2.*(end+1-start)))
    #print 'Loss value (percentage):', loss
    return  loss

In [10]:
#midi_file = "/Users/robin/Desktop/CS221_projet/Open_MIDI_file/ode.mid"
midi_file = "/Users/charlesburlin/courses/cs221/proj/test/ode.mid"
mypattern = dataUtil.MusicPattern(midi_file)
mypattern.midiToLisp()
mypattern.lispToPrim()
mypattern.primToLisp()
mypattern.lispToMidi()


Loaded /Users/charlesburlin/courses/cs221/proj/test/ode.mid


In [11]:
trackNum = 0
weight_uni = 0.5
weight_bi = 2
weight_tri = 1
window = (25, 50)

primReconstruction = reconstruction(mypattern,  window, weight_uni, weight_bi, weight_tri, trackNum=0, bi=True, tri=False, rhythmRec=False, bi_gap=False)
primOriginal = mypattern.primTracks[trackNum]
Loss = lossFunction(window, primOriginal, primReconstruction)
Loss

Position: 27 Original pitch: 74 Reconstructed pitch: 76
Position: 28 Original pitch: 74 Reconstructed pitch: 76
Position: 29 Original pitch: 74 Reconstructed pitch: 76
Position: 30 Original pitch: 74 Reconstructed pitch: 76
Position: 31 Original pitch: 74 Reconstructed pitch: 76
Position: 36 Original pitch: 77 Reconstructed pitch: 76
Position: 37 Original pitch: 77 Reconstructed pitch: 76
Position: 38 Original pitch: 79 Reconstructed pitch: 76
Position: 39 Original pitch: 79 Reconstructed pitch: 76
Position: 40 Original pitch: 79 Reconstructed pitch: 76
Position: 41 Original pitch: 79 Reconstructed pitch: 76
Position: 42 Original pitch: 77 Reconstructed pitch: 76
Position: 43 Original pitch: 77 Reconstructed pitch: 76
Position: 46 Original pitch: 74 Reconstructed pitch: 76
Position: 47 Original pitch: 74 Reconstructed pitch: 76
Position: 48 Original pitch: 72 Reconstructed pitch: 74
Position: 49 Original pitch: 72 Reconstructed pitch: 74


32.692307692307686

In [ ]:

import sys
import numpy as np
log = open("log.txt", "w")
sys.stdout = log

#path = '/Users/robin/Desktop/CS221_projet/GitHub/cs221/dat/bach-chorales/chorales.lisp'
path = "/Users/charlesburlin/courses/cs221/proj/cs221/dat/bach-chorales/chorales.lisp"
weight_uni = 1
weight_bi = 1
weight_tri = 1

results = {}

for chorale in range(100):
    print chorale
    trackNum=0
    bi = True
    rhythmRec = True
    mypattern = dataUtil.BachChorale(path, chorale)
    L = len(mypattern.primTracks[0])
    window_list = [(L/4, L/4 +5), (L/4, L/4 +10), (L/2, L/2 +5), (L/2, L/2 +10), (3*L/4, 3*L/4 +5)]
    if len(mypattern.primTracks[0]) > 3*L/4 +10:
        window_list.append((3*L/4, 3*L/4 +10))
    #if len(mypattern.primTracks[0]) > L/2 +20:
    #    window_list.append((L/2, L/2 +20))
    #if len(mypattern.primTracks[0]) > 3*L/4 +20:
    #    window_list.append((3*L/4, 3*L/4 +20))
    for window in window_list:
        for tri in [True, False]:
            for bi_gap in [True, False]:
                primReconstruction = reconstruction(mypattern, window, weight_uni, weight_bi, weight_tri, trackNum, bi, tri, rhythmRec, bi_gap)
                primOriginal = mypattern.primTracks[trackNum]
                Loss = lossFunction(window, primOriginal, primReconstruction)
                results[(chorale, L, window, tri, bi_gap)] = Loss
                print 'chorale numero: ', chorale
                print 'chorale length: ', L
                print 'window: ', window
                print 'tri: ', tri
                print 'bi_gap: ', bi_gap
                print 'Loss: ', Loss
                print '########################'

# Save             
np.save('results_dict.npy', results) 



In [ ]:
import sys
import numpy as np
log = open("laplace_log.txt", "w")
sys.stdout = log

#path = '/Users/robin/Desktop/CS221_projet/GitHub/cs221/dat/bach-chorales/chorales.lisp'
path = "/Users/charlesburlin/courses/cs221/proj/cs221/dat/bach-chorales/chorales.lisp"
weight_uni = 1
weight_bi = 1
weight_tri = 1

results = {}
blocked = [(12, 1)]
for chorale in range(0, 100):
    trackNum=0
    bi = True
    tri = True
    rhythmRec = True
    bi_gap = False
    mypattern = dataUtil.BachChorale(path, chorale)
    L = len(mypattern.primTracks[0])
    window = (L/2, L/2 +5)
    laplace_coefs = [0, 1, 5]
    for laplace_coef in laplace_coefs:
        try:
            primReconstruction = reconstruction(mypattern, window, weight_uni, weight_bi, weight_tri, trackNum, bi, tri, rhythmRec, bi_gap, laplace_coef)
            primOriginal = mypattern.primTracks[trackNum]
            Loss = lossFunction(window, primOriginal, primReconstruction)
            results[(chorale, L, window, laplace_coef)] = Loss
            print 'chorale numero: ', chorale
            print 'chorale length: ', L
            print 'window: ', window
            print 'laplace: ', laplace_coef
            print 'tri: ', tri
            print 'bi_gap: ', bi_gap
            print 'Loss: ', Loss
            print '########################'
            sys.stdout.flush()
        except:
            pass
# Save             
np.save('results_dict_laplace.npy', results) 

In [ ]:
import sys
import numpy as np
log = open("weight_log.txt", "w")
sys.stdout = log

#path = '/Users/robin/Desktop/CS221_projet/GitHub/cs221/dat/bach-chorales/chorales.lisp'
path = "/Users/charlesburlin/courses/cs221/proj/cs221/dat/bach-chorales/chorales.lisp"
weight_uni = 1
weight_bi = 1
weight_tri = 1

results = {}
weights = [(1, 1, 1), (1, 5, 10), (10, 5, 1), (1, 10, 5), (5, 1, 10), (5, 10, 1)]

for chorale in range(100):
    trackNum=0
    laplace = 0
    bi = True
    rhythmRec = True
    bi_gap = False
    mypattern = dataUtil.BachChorale(path, chorale)
    L = len(mypattern.primTracks[0])
    window = (L/2, L/2 +5)

    for weight in weights:
        try:
            weight_uni, weight_bi, weight_tri = weight
            primReconstruction = reconstruction(mypattern, window, weight_uni, weight_bi, weight_tri, trackNum, bi, tri, rhythmRec, bi_gap, 0)
            primOriginal = mypattern.primTracks[trackNum]
            Loss = lossFunction(window, primOriginal, primReconstruction)
            results[(chorale, L, window, weight)] = Loss
            print 'chorale numero: ', chorale
            print 'weights', weight
            print 'chorale length: ', L
            print 'window: ', window
            print 'laplace: ', laplace_coef
            print 'tri: ', tri
            print 'bi_gap: ', bi_gap
            print 'Loss: ', Loss
            print '########################'
        except:
            pass
# Save             
np.save('results_dict_weights.npy', results) 

## Plot Data Analysis

In [88]:
# Load
read_dictionary = np.load('results_dict_laplace.npy').item()
read_dictionary

{(0, 80, (40, 45), 0): 91.66666666666666,
 (0, 80, (40, 45), 1): 91.66666666666666,
 (0, 80, (40, 45), 5): 91.66666666666666,
 (1, 126, (63, 68), 0): 25.0,
 (1, 126, (63, 68), 1): 25.0,
 (1, 126, (63, 68), 5): 25.0,
 (2, 104, (52, 57), 0): 16.666666666666664,
 (2, 104, (52, 57), 1): 16.666666666666664,
 (2, 104, (52, 57), 5): 50.0,
 (3, 160, (80, 85), 0): 66.66666666666667,
 (3, 160, (80, 85), 1): 50.0,
 (3, 160, (80, 85), 5): 50.0,
 (4, 80, (40, 45), 0): 66.66666666666667,
 (4, 80, (40, 45), 1): 66.66666666666667,
 (4, 80, (40, 45), 5): 66.66666666666667,
 (5, 136, (68, 73), 0): 41.666666666666664,
 (5, 136, (68, 73), 1): 41.666666666666664,
 (5, 136, (68, 73), 5): 41.666666666666664,
 (6, 31, (15, 20), 0): 58.33333333333333,
 (6, 31, (15, 20), 1): 58.33333333333333,
 (6, 31, (15, 20), 5): 58.33333333333333,
 (7, 222, (111, 116), 0): 41.666666666666664,
 (7, 222, (111, 116), 1): 8.333333333333337,
 (7, 222, (111, 116), 5): 8.333333333333337,
 (8, 160, (80, 85), 0): 33.333333333333336,

In [89]:
read_dictionary = np.load('results_dict_weights.npy').item()
read_dictionary

{(0, 80, (40, 45), (1, 1, 1)): 91.66666666666666,
 (0, 80, (40, 45), (1, 5, 10)): 91.66666666666666,
 (0, 80, (40, 45), (1, 10, 5)): 91.66666666666666,
 (0, 80, (40, 45), (5, 1, 10)): 91.66666666666666,
 (0, 80, (40, 45), (5, 10, 1)): 91.66666666666666,
 (0, 80, (40, 45), (10, 5, 1)): 91.66666666666666,
 (1, 126, (63, 68), (1, 1, 1)): 25.0,
 (1, 126, (63, 68), (1, 5, 10)): 25.0,
 (1, 126, (63, 68), (1, 10, 5)): 25.0,
 (1, 126, (63, 68), (5, 1, 10)): 25.0,
 (1, 126, (63, 68), (5, 10, 1)): 25.0,
 (1, 126, (63, 68), (10, 5, 1)): 25.0,
 (2, 104, (52, 57), (1, 1, 1)): 16.666666666666664,
 (2, 104, (52, 57), (1, 5, 10)): 16.666666666666664,
 (2, 104, (52, 57), (1, 10, 5)): 16.666666666666664,
 (2, 104, (52, 57), (5, 1, 10)): 16.666666666666664,
 (2, 104, (52, 57), (5, 10, 1)): 16.666666666666664,
 (2, 104, (52, 57), (10, 5, 1)): 16.666666666666664,
 (3, 160, (80, 85), (1, 1, 1)): 66.66666666666667,
 (3, 160, (80, 85), (1, 5, 10)): 66.66666666666667,
 (3, 160, (80, 85), (1, 10, 5)): 66.666666

### Only Binary factors and different windows

In [4]:
results_bi_w_1 = {}
results_bi_w_2 = {}
results_bi_w_3 = {}
results_bi_w_4 = {}
results_bi_w_5 = {}
results_bi_w_6 = {}
for k in read_dictionary.keys():
    if k[3] == False and k[4] == False:
        if (k[2][0] == k[1]/4) and (k[2][1] == k[1]/4 + 5):
             results_bi_w_1[k[0]] = read_dictionary[k]
        if (k[2][0] == k[1]/4) and (k[2][1] == k[1]/4 + 10):
             results_bi_w_2[k[0]] = read_dictionary[k]
        if (k[2][0] == k[1]/2) and (k[2][1] == k[1]/2 + 5):
             results_bi_w_3[k[0]] = read_dictionary[k]
        if (k[2][0] == k[1]/2) and (k[2][1] == k[1]/2 + 10):
             results_bi_w_4[k[0]] = read_dictionary[k]
        if (k[2][0] == 3*k[1]/4) and (k[2][1] == 3*k[1]/4 + 5):
             results_bi_w_5[k[0]] = read_dictionary[k]
        if (k[2][0] == 3*k[1]/4) and (k[2][1] == 3*k[1]/4 + 10):
             results_bi_w_6[k[0]] = read_dictionary[k]

In [141]:
list_results_bi_w_1 = sorted(results_bi_w_1.items()) # sorted by key, return a list of tuples
list_results_bi_w_2 = sorted(results_bi_w_2.items()) # sorted by key, return a list of tuples
list_results_bi_w_3 = sorted(results_bi_w_3.items()) # sorted by key, return a list of tuples
list_results_bi_w_4 = sorted(results_bi_w_4.items()) # sorted by key, return a list of tuples
list_results_bi_w_5 = sorted(results_bi_w_5.items()) # sorted by key, return a list of tuples
list_results_bi_w_6 = sorted(results_bi_w_6.items()) # sorted by key, return a list of tuples


x_1, y_1 = zip(*list_results_bi_w_1) # unpack a list of pairs into two tuples
x_2, y_2 = zip(*list_results_bi_w_2) # unpack a list of pairs into two tuples
x_3, y_3 = zip(*list_results_bi_w_3) # unpack a list of pairs into two tuples
x_4, y_4 = zip(*list_results_bi_w_4) # unpack a list of pairs into two tuples
x_5, y_5 = zip(*list_results_bi_w_5) # unpack a list of pairs into two tuples
x_6, y_6 = zip(*list_results_bi_w_6) # unpack a list of pairs into two tuples



#plt.plot(x_1, y_1, color='#047495', label = 'Window 1'), plt.plot(x_1, np.mean(y_1)*np.ones(len(y_1)), '--', color='#047495')
plt.plot(x_2, y_2, color='#fc5a50', label = 'Window 2'), plt.plot(x_2, np.mean(y_2)*np.ones(len(y_2)), '--', color='#fc5a50')
#plt.plot(x_3, y_3, color='#fcb001', label = 'Window 3'), plt.plot(x_3, np.mean(y_3)*np.ones(len(y_3)), '--', color='#fcb001')
plt.plot(x_4, y_4, color='#9d5783', label = 'Window 4'), plt.plot(x_4, np.mean(y_4)*np.ones(len(y_4)), '--', color='#9d5783')
#plt.plot(x_5, y_5, color='#9d7651', label = 'Window 5'), plt.plot(x_5, np.mean(y_5)*np.ones(len(y_5)), '--', color='#9d7651')
plt.plot(x_6, y_6, color='#77ab56', label = 'Window 6'), plt.plot(x_6, np.mean(y_6)*np.ones(len(y_6)), '--', color='#77ab56')

plt.legend(bbox_to_anchor=(1.05, 1), loc=2, borderaxespad=0.)
plt.xlabel('Bach Chorale Number')
plt.ylabel('Loss (%)')
fig = plt.gcf()
fig.set_size_inches(20, 10)

#plt.show()
fig.savefig('window_2_4_6.png', dpi=100, bbox_inches='tight')

### Binary, Trinary, Binary with Gap and all factors, averaged over all windows

In [6]:
results_bi = {}
results_tri = {}
results_bi_gap = {}
results_all = {}

results_bi_temp = {}
results_tri_temp = {}
results_bi_gap_temp = {}
results_all_temp = {}

for k in read_dictionary.keys():
    if k[3] == False and k[4] == False:
        if k[0] in results_bi_temp.keys():
            results_bi_temp[k[0]].append(read_dictionary[k])
        else:
            results_bi_temp[k[0]] = [read_dictionary[k]]
    if k[3] == True and k[4] == False:
        if k[0] in results_tri_temp.keys():
            results_tri_temp[k[0]].append(read_dictionary[k])
        else:
            results_tri_temp[k[0]] = [read_dictionary[k]]
    if k[3] == False and k[4] == True:
        if k[0] in results_bi_gap_temp.keys():
            results_bi_gap_temp[k[0]].append(read_dictionary[k])
        else:
            results_bi_gap_temp[k[0]] = [read_dictionary[k]]
    if k[3] == True and k[4] == True:
        if k[0] in results_all_temp.keys():
            results_all_temp[k[0]].append(read_dictionary[k])
        else:
            results_all_temp[k[0]] = [read_dictionary[k]]

for k in results_bi_temp.keys():
    results_bi[k] = np.mean(results_bi_temp[k])
for k in results_tri_temp.keys():
    results_tri[k] = np.mean(results_tri_temp[k])
for k in results_bi_gap_temp.keys():
    results_bi_gap[k] = np.mean(results_bi_gap_temp[k])
for k in results_all_temp.keys():
    results_all[k] = np.mean(results_all_temp[k])   

In [130]:
list_results_bi = sorted(results_bi.items()) # sorted by key, return a list of tuples
list_results_tri = sorted(results_tri.items()) # sorted by key, return a list of tuples
list_results_bi_gap = sorted(results_bi_gap.items()) # sorted by key, return a list of tuples
list_results_all = sorted(results_all.items()) # sorted by key, return a list of tuples

x_1, y_1 = zip(*list_results_bi) # unpack a list of pairs into two tuples
x_2, y_2 = zip(*list_results_tri) # unpack a list of pairs into two tuples
x_3, y_3 = zip(*list_results_bi_gap) # unpack a list of pairs into two tuples
x_4, y_4 = zip(*list_results_all) # unpack a list of pairs into two tuples


plt.plot(x_1, y_1, color='#047495', label = 'Only Binary factors'), plt.plot(x_1, np.mean(y_1)*np.ones(len(y_1)), '--', color='#047495')
#plt.plot(x_2, y_2, color='#fc5a50', label = 'Binary & Trinary factors'), plt.plot(x_2, np.mean(y_2)*np.ones(len(y_2)), '--', color='#fc5a50')
plt.plot(x_3, y_3, color='#fcb001', label = 'Binary & Binary with gaps factors'), plt.plot(x_3, np.mean(y_3)*np.ones(len(y_3)), '--', color='#fcb001')
#plt.plot(x_4, y_4, color='#9d5783', label = 'All factors'), plt.plot(x_4, np.mean(y_4)*np.ones(len(y_4)), '--', color='#9d5783')

plt.legend(bbox_to_anchor=(1.05, 1), loc=2, borderaxespad=0.)
plt.xlabel('Bach Chorale Number')
plt.ylabel('Loss (%)')
fig = plt.gcf()
fig.set_size_inches(20, 10)

#plt.show()
fig.savefig('bi_bi_gap.png', dpi=100, bbox_inches='tight')

### Binary, Trinary, Binary with Gap and all factors, averaged over all Bach chorales

In [8]:
results_bi_bar = {}
results_tri_bar = {}
results_bi_gap_bar = {}
results_all_bar = {}

results_bi_bar_temp = {}
results_tri_bar_temp = {}
results_bi_gap_bar_temp = {}
results_all_bar_temp = {}

for k in read_dictionary.keys():
    if k[3] == False and k[4] == False:
        results_bi_bar_temp[(k[0], k[1], k[2])] = read_dictionary[k]
    if k[3] == True and k[4] == False:
        results_tri_bar_temp[(k[0], k[1], k[2])] = read_dictionary[k]
    if k[3] == False and k[4] == True:
        results_bi_gap_bar_temp[(k[0], k[1], k[2])] = read_dictionary[k]
    if k[3] == True and k[4] == True:
        results_all_bar_temp[(k[0], k[1], k[2])] = read_dictionary[k]

for k in results_bi_bar_temp.keys():
    if (k[2][0] == k[1]/4) and (k[2][1] == k[1]/4 + 5):
        if 1 in results_bi_bar.keys():
            results_bi_bar[1].append(results_bi_bar_temp[k])
        else:
            results_bi_bar[1]= [results_bi_bar_temp[k]]
    if (k[2][0] == k[1]/4) and (k[2][1] == k[1]/4 + 10):
        if 2 in results_bi_bar.keys():
            results_bi_bar[2].append(results_bi_bar_temp[k])
        else:
            results_bi_bar[2]= [results_bi_bar_temp[k]]
    if (k[2][0] == k[1]/2) and (k[2][1] == k[1]/2 + 5):
        if 3 in results_bi_bar.keys():
            results_bi_bar[3].append(results_bi_bar_temp[k])
        else:
            results_bi_bar[3]= [results_bi_bar_temp[k]]
    if (k[2][0] == k[1]/2) and (k[2][1] == k[1]/2 + 10):
        if 4 in results_bi_bar.keys():
            results_bi_bar[4].append(results_bi_bar_temp[k])
        else:
            results_bi_bar[4]= [results_bi_bar_temp[k]]
    if (k[2][0] == 3*k[1]/4) and (k[2][1] == 3*k[1]/4 + 5):
        if 5 in results_bi_bar.keys():
            results_bi_bar[5].append(results_bi_bar_temp[k])
        else:
            results_bi_bar[5]= [results_bi_bar_temp[k]]
    if (k[2][0] == 3*k[1]/4) and (k[2][1] == 3*k[1]/4 + 10):
        if 6 in results_bi_bar.keys():
            results_bi_bar[6].append(results_bi_bar_temp[k])
        else:
            results_bi_bar[6]= [results_bi_bar_temp[k]]
            
for k in results_tri_bar_temp.keys():
    if (k[2][0] == k[1]/4) and (k[2][1] == k[1]/4 + 5):
        if 1 in results_tri_bar.keys():
            results_tri_bar[1].append(results_tri_bar_temp[k])
        else:
            results_tri_bar[1]= [results_tri_bar_temp[k]]
    if (k[2][0] == k[1]/4) and (k[2][1] == k[1]/4 + 10):
        if 2 in results_tri_bar.keys():
            results_tri_bar[2].append(results_tri_bar_temp[k])
        else:
            results_tri_bar[2]= [results_tri_bar_temp[k]]
    if (k[2][0] == k[1]/2) and (k[2][1] == k[1]/2 + 5):
        if 3 in results_tri_bar.keys():
            results_tri_bar[3].append(results_tri_bar_temp[k])
        else:
            results_tri_bar[3]= [results_tri_bar_temp[k]]
    if (k[2][0] == k[1]/2) and (k[2][1] == k[1]/2 + 10):
        if 4 in results_tri_bar.keys():
            results_tri_bar[4].append(results_tri_bar_temp[k])
        else:
            results_tri_bar[4]= [results_tri_bar_temp[k]]
    if (k[2][0] == 3*k[1]/4) and (k[2][1] == 3*k[1]/4 + 5):
        if 5 in results_tri_bar.keys():
            results_tri_bar[5].append(results_tri_bar_temp[k])
        else:
            results_tri_bar[5]= [results_tri_bar_temp[k]]
    if (k[2][0] == 3*k[1]/4) and (k[2][1] == 3*k[1]/4 + 10):
        if 6 in results_tri_bar.keys():
            results_tri_bar[6].append(results_tri_bar_temp[k])
        else:
            results_tri_bar[6]= [results_tri_bar_temp[k]]
            
for k in results_bi_gap_bar_temp.keys():
    if (k[2][0] == k[1]/4) and (k[2][1] == k[1]/4 + 5):
        if 1 in results_bi_gap_bar.keys():
            results_bi_gap_bar[1].append(results_bi_gap_bar_temp[k])
        else:
            results_bi_gap_bar[1]= [results_bi_gap_bar_temp[k]]
    if (k[2][0] == k[1]/4) and (k[2][1] == k[1]/4 + 10):
        if 2 in results_bi_gap_bar.keys():
            results_bi_gap_bar[2].append(results_bi_gap_bar_temp[k])
        else:
            results_bi_gap_bar[2]= [results_bi_gap_bar_temp[k]]
    if (k[2][0] == k[1]/2) and (k[2][1] == k[1]/2 + 5):
        if 3 in results_bi_gap_bar.keys():
            results_bi_gap_bar[3].append(results_bi_gap_bar_temp[k])
        else:
            results_bi_gap_bar[3]= [results_bi_gap_bar_temp[k]]
    if (k[2][0] == k[1]/2) and (k[2][1] == k[1]/2 + 10):
        if 4 in results_bi_gap_bar.keys():
            results_bi_gap_bar[4].append(results_bi_gap_bar_temp[k])
        else:
            results_bi_gap_bar[4]= [results_bi_gap_bar_temp[k]]
    if (k[2][0] == 3*k[1]/4) and (k[2][1] == 3*k[1]/4 + 5):
        if 5 in results_bi_gap_bar.keys():
            results_bi_gap_bar[5].append(results_bi_gap_bar_temp[k])
        else:
            results_bi_gap_bar[5]= [results_bi_gap_bar_temp[k]]
    if (k[2][0] == 3*k[1]/4) and (k[2][1] == 3*k[1]/4 + 10):
        if 6 in results_bi_gap_bar.keys():
            results_bi_gap_bar[6].append(results_bi_gap_bar_temp[k])
        else:
            results_bi_gap_bar[6]= [results_bi_gap_bar_temp[k]]
            
for k in results_all_bar_temp.keys():
    if (k[2][0] == k[1]/4) and (k[2][1] == k[1]/4 + 5):
        if 1 in results_all_bar.keys():
            results_all_bar[1].append(results_all_bar_temp[k])
        else:
            results_all_bar[1]= [results_all_bar_temp[k]]
    if (k[2][0] == k[1]/4) and (k[2][1] == k[1]/4 + 10):
        if 2 in results_all_bar.keys():
            results_all_bar[2].append(results_all_bar_temp[k])
        else:
            results_all_bar[2]= [results_all_bar_temp[k]]
    if (k[2][0] == k[1]/2) and (k[2][1] == k[1]/2 + 5):
        if 3 in results_all_bar.keys():
            results_all_bar[3].append(results_all_bar_temp[k])
        else:
            results_all_bar[3]= [results_all_bar_temp[k]]
    if (k[2][0] == k[1]/2) and (k[2][1] == k[1]/2 + 10):
        if 4 in results_all_bar.keys():
            results_all_bar[4].append(results_all_bar_temp[k])
        else:
            results_all_bar[4]= [results_all_bar_temp[k]]
    if (k[2][0] == 3*k[1]/4) and (k[2][1] == 3*k[1]/4 + 5):
        if 5 in results_all_bar.keys():
            results_all_bar[5].append(results_all_bar_temp[k])
        else:
            results_all_bar[5]= [results_all_bar_temp[k]]
    if (k[2][0] == 3*k[1]/4) and (k[2][1] == 3*k[1]/4 + 10):
        if 6 in results_all_bar.keys():
            results_all_bar[6].append(results_all_bar_temp[k])
        else:
            results_all_bar[6]= [results_all_bar_temp[k]]
            
for k in results_bi_bar.keys():
    results_bi_bar[k] = np.mean(results_bi_bar[k])
for k in results_tri_bar.keys():
    results_tri_bar[k] = np.mean(results_tri_bar[k])
for k in results_bi_gap_bar.keys():
    results_bi_gap_bar[k] = np.mean(results_bi_gap_bar[k])
for k in results_all_bar.keys():
    results_all_bar[k] = np.mean(results_all_bar[k])

In [121]:
X = np.arange(len(results_bi_bar))
ax = plt.subplot(111)
ax.bar(X-0.3, results_bi_bar.values(), width=0.2, color='#047495', align='center')
ax.bar(X-0.1, results_tri_bar.values(), width=0.2, color='#fc5a50', align='center')
ax.bar(X+0.1, results_bi_gap_bar.values(), width=0.2, color='#fcb001', align='center')
ax.bar(X+0.3, results_all_bar.values(), width=0.2, color='#9d5783', align='center')



ax.legend(('Binary factors only','Binary & Trinary factors', 'Binary & Binary with gaps factors', 'All factors'), bbox_to_anchor=(1.05, 1), loc=2)
plt.xticks(X, results_bi_bar.keys())
plt.title("Loss obtained with various factors", fontsize=17)
plt.xlabel('Window')
plt.ylabel('Loss (%)')
fig = plt.gcf()
fig.set_size_inches(20, 10)

#plt.show()
fig.savefig('bars.png', dpi=100, bbox_inches='tight')

In [143]:
results_bi_bar

{1: 45.916666666666657,
 2: 44.727272727272727,
 3: 44.916666666666657,
 4: 46.499999999999993,
 5: 48.333333333333343,
 6: 46.285434995112418}

In [144]:
results_tri_bar

{1: 48.148148148148145,
 2: 47.86931818181818,
 3: 51.262626262626263,
 4: 51.716141001855284,
 5: 56.292517006802726,
 6: 51.075268817204282}

In [145]:
results_bi_gap_bar

{1: 47.416666666666657,
 2: 49.494949494949495,
 3: 49.416666666666671,
 4: 47.727272727272727,
 5: 50.666666666666671,
 6: 48.729227761485831}

In [146]:
results_all_bar

{1: 47.053872053872041,
 2: 48.910984848484851,
 3: 50.589225589225585,
 4: 52.690166975881262,
 5: 53.656462585034014,
 6: 52.248289345063526}

# Finding optimal weights for unary and binary factors

In [ ]:
trackNum = 0
weight_uni = 1
weight_bi = 1
#window = (20, 30)
for i in range(10,30):
    window = (i, i+10)
    primReconstruction = reconstruction(mypattern,  window, weight_uni, weight_bi, trackNum=0, bi=True, tri=False, rhythmRec=False)
    primOriginal = mypattern.primTracks[trackNum]
    Loss = lossFunction(window, primOriginal, primReconstruction)
    print Loss

In [ ]:
Loss_mean = []
for i in range(len(Weight_factors)):
    weight_uni = Weight_factors[i]
    for j in range(len(Weight_factors)):
        weight_bi = Weight_factors[j]
        Losses = [trainExamples[k][1] for k in range(len(trainExamples)) if (trainExamples[k][0][1] == weight_uni) & (trainExamples[k][0][2] == weight_bi)]
        Loss_mean.append([weight_uni, weight_bi, sum(Losses)/len(Losses)])

In [ ]:
min([Loss_mean[i][2] for i in range(len(Loss_mean))])

In [ ]:
[Loss_mean[i][2] for i in range(len(Loss_mean))]